# Infiltration into soils using the green ampt model

In [17]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
from ipywidgets import interact, interactive, fixed, interact_manual, Button, HBox, VBox
from ipywidgets import interactive_output
from IPython.display import clear_output


The Green Ampt Model

$$t = \frac{i_t - \Delta\theta \Delta h \ln(1 + \frac{i_t}{\Delta\theta \Delta h})}{K_s}$$

where $t$ is the time, $i_t$ is the cumulative infiltration, $\Delta\theta$ is the difference between the initial and wetted soil water content, $\Delta h$ is the difference between the initial and final soil water potential, and $K_s$ is the saturated hydraulic conductivity of the soil.

In [18]:
def t_for_cum_inf(i_t, d_theta, d_h, Ks):
    return (i_t - d_theta * d_h * np.log(1+ (i_t/(d_theta*d_h))))/Ks

def short_time_approximation(d_theta, d_h, K, t):
    """Short time approximation for cumulative infiltration
    Parameters
    ----------
    d_theta : float
        Change in volumetric water content
    d_h : float
        Change in pressure head
    K : float
        Hydraulic conductivity
    t : array_like
        Time
    Returns
    -------
    i_t : array
        Cumulative infiltration"""

    return np.sqrt((d_theta*d_h*K)/(2*t))

def long_time_approximation(K, x):
    """Long time approximation for cumulative infiltration
    Parameters
    ----------
    K : float
        Hydraulic conductivity
    t : array_like
        Time
    Returns
    -------
    K : float
        Hydraulic conductivity"""
    
    it_long = np.array([K]*len(x))

    return it_long

def d_h(h0, hf):
    return h0 - hf

def d_theta(theta_0, theta_i):
    return theta_0 - theta_i

In [19]:
h0 = 0
theta_0 = 0.44

# why is t getting negativ  with higher hf?
when theta_i is higher than theta_0

In [ ]:
# List to store data for the short time approximation reference lines
reference_lines = []
ref_line_params = []
short_time_ref = []
short_time_params = []

def plot_scenario_1(hf=-60, theta_i=0.2, Ks=4):
    delta_theta = d_theta(theta_0, theta_i)
    delta_h = d_h(h0, hf)
    
    I_t = np.arange(0, 260, 0.1)
    t = t_for_cum_inf(I_t, delta_theta, delta_h, Ks)
    # write the time to a file
    np.savetxt('time.txt', t, delimiter=',')
    new_x = np.arange(round(t[0], 1), round(t[-1], 1), 0.01)
    y2 = np.interp(new_x, t, I_t)
    diff = np.diff(y2)
    
    i_t_short = short_time_approximation(delta_theta, delta_h, Ks, new_x)
    i_t_long = long_time_approximation(Ks, new_x)

    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Plot all reference lines for the short time approximation
    for ref, params in zip(reference_lines, ref_line_params):
        ref_x, ref_y = ref
        ax.plot(ref_x, ref_y, linestyle='-.', alpha=0.6, label='hf:{} theta_i:{} Ks:{}'.format(params[0], params[1], params[2]))
    
    ax.plot(new_x[1:-5], diff[:-5]*100, color='black', label="theta_i=" + str(theta_i) + ", hf=" + str(hf))
    ax.plot(new_x, i_t_short, 'orange', linestyle='-.', label='Short time approximation')
    ax.plot(new_x, i_t_long, 'red', linestyle='--', label='Long time approximation')
    
    ax.set_ylim(0.1, 140)
    ax.set_xlim(-5, max(new_x))
    plt.yscale('log')
    ax.set_xlabel('Time [h]', fontsize=12)
    ax.set_ylabel('infiltration rate [cm/h]', fontsize=12)
    
    # Move the legend outside of the plot
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    
    plt.show()

def on_button_click(button):
    # Current parameter values
    hf_val = hf_slider.value
    theta_i_val = theta_i_slider.value
    Ks_val = Ks_slider.value
    
    delta_theta = d_theta(theta_0, theta_i_val)
    delta_h = d_h(h0, hf_val)
    
    I_t = np.arange(0, 260, 0.1)
    t = t_for_cum_inf(I_t, delta_theta, delta_h, Ks_val)
    print(t)
    new_x = np.arange(round(t[0], 1), round(t[-1], 1), 0.01)
    i_t_short = short_time_approximation(delta_theta, delta_h, Ks_val, new_x)
    
    # Store the current short time approximation data as a reference line
    reference_lines.append((new_x, i_t_short))
    ref_line_params.append((hf_val, theta_i_val, Ks_val))
    short_time_ref.append((new_x, i_t_short))
    short_time_params.append((hf_val, theta_i_val, Ks_val))
    
    clear_output(wait=True)
    display(interactive_plot)
    display(button)


# Create the button
button = Button(description="Fixate Short time approximation")
button.on_click(on_button_click)

# Create sliders for the parameters
hf_slider = widgets.FloatSlider(value=-60, min=-200, max=0, step=1, description='hf:')
theta_i_slider = widgets.FloatSlider(value=0.2, min=0, max=1, step=0.01, description='theta_i:')
Ks_slider = widgets.FloatSlider(value=4, min=0, max=10, step=0.1, description='Ks:')

# Display the interactive plot with the sliders and button
interactive_plot = interactive(plot_scenario_1, hf=hf_slider, theta_i=theta_i_slider, Ks=Ks_slider)
display(interactive_plot)
display(button)